<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/13YIbphbWc62sfa-bCh8MLQWKizaXbQK9/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

In [ ]:
import pandas as pd

df = pd.read_excel("/content/data_problems.xlsx", index_col=0)
df.head()

,Задача,Тема
0,Между девятью планетами Солнечной системы введ...,Графы
1,"В стране Цифра есть 9 городов с названиями 1, ...",Графы
2,"В государстве 100 городов, и из каждого из них...",Графы
3,"В классе 30 человек. Может ли быть так, что 9 ...",Графы
4,В городе Маленьком 15 телефонов. Можно ли их с...,Графы


In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel

# Кастомный Dataset
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, label_to_id, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.label_to_id = label_to_id
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label_str = self.labels[idx]
        label = self.label_to_id[label_str]  # Преобразуем строковую метку в число

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def prepare_data(df, train_df, val_df, model_name, feat_name, target, max_length=512):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    unique_labels = sorted(df[target].unique())
    label_to_id = {label: i for i, label in enumerate(unique_labels)}
    id_to_label = {i: label for label, i in label_to_id.items()}

    train_dataset = TextClassificationDataset(
        train_df[feat_name].tolist(),
        train_df[target].tolist(),
        tokenizer,
        label_to_id,
        max_length
    )

    val_dataset = TextClassificationDataset(
        val_df[feat_name].tolist(),
        val_df[target].tolist(),
        tokenizer,
        label_to_id,
        max_length
    )

    return train_dataset, val_dataset, label_to_id, id_to_label, tokenizer

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [ ]:
### This is just an interface example. You may change it if you want.
import torch.nn as nn
from transformers import AutoModel, AutoConfig

class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model, dropout, num_classes):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_transformer_model) #...
        # YOUR CODE: create additional layers for classfication
        hidden_size = self.backbone.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        # YOUR CODE: propagate inputs through the model. Return dict with logits
        transformer_out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        outputs = transformer_out.last_hidden_state[:, 0]  # [CLS] токен
        logits = self.classifier(self.dropout(outputs))

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        # Возвращаем словарь как ожидает Trainer
        return {
            "loss": loss,
            "logits": logits
        }

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel):
    for param in model.backbone.parameters():
        param.requires_grad = False
    return model

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [ ]:
import copy
from transformers import Trainer, TrainingArguments

def train_transformer(transformer_model, train_dataset, val_dataset, freeze_backbone):
    model = copy.deepcopy(transformer_model)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        eval_strategy="steps",
        eval_steps=150,
        logging_steps=150
    )

    if freeze_backbone:
        freeze_backbone_function(model)

    # Создаем Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Запускаем обучение
    trainer.train()

    # Возвращаем обученную модель
    return trainer.model

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
model_name = "cointegrated/rubert-tiny2"
train_dataset, val_dataset, label_to_id, id_to_label, tokenizer = prepare_data(
    df, train_df, val_df, model_name, "Задача", "Тема"
)
num_classes = len(label_to_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
rubert_tiny_transformer_model_1 = TransformerClassificationModel(model_name, dropout=0.2, num_classes=num_classes)
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model_1, train_dataset, val_dataset,
                                                                freeze_backbone=True)

rubert_tiny_transformer_model_2 = TransformerClassificationModel(model_name, dropout=0.2, num_classes=num_classes)
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model_2, train_dataset, val_dataset,
                                                                freeze_backbone=False)

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
df_eng = pd.read_csv("/content/data_problems_translated.xlsx - Sheet1.csv", index_col=0)
train_df_eng, val_df_eng = train_test_split(df_eng, test_size=0.2)
df_eng.head()

,problem_text,topic
0,To prove that the sum of the numbers of the ex...,number_theory
1,( b) Will the statement of the previous challe...,number_theory
2,The quadratic three-member graph with the coef...,polynoms
3,Can you draw on the surface of Rubik's cube a ...,combinatorics
4,"Dima, who came from Vrunlandia, said that ther...",graphs


In [ ]:
model_name_eng = "tbs17/MathBert"
train_dataset_eng, val_dataset_eng, label_to_id_eng, id_to_label_eng, tokenizer_eng = prepare_data(
    df_eng, train_df_eng, val_df_eng, model_name_eng, "problem_text", "topic"
)
num_classes = len(label_to_id)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
### YOUR CODE IS HERE (probably, similar on the previous step)
MathBert_transformer_model_1 = TransformerClassificationModel(model_name_eng, dropout=0.2, num_classes=num_classes)
MathBert_finetuned_with_freezed_backbone = train_transformer(MathBert_transformer_model_1, train_dataset_eng, val_dataset_eng,
                                                                freeze_backbone=True)

MathBert_transformer_model_2 = TransformerClassificationModel(model_name_eng, dropout=0.2, num_classes=num_classes)
MathBert_full_finetuned = train_transformer(MathBert_transformer_model_2, train_dataset_eng, val_dataset_eng,
                                                                freeze_backbone=False)

Тут уже в конце переобучилась моделька...

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
!pip install bertviz -q

In [ ]:
from bertviz import head_view
from bertviz import model_view

def bertviz_plot(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.backbone(**inputs, output_attentions=True)

    # Получение матрицы внимания
    attention = outputs.attentions[0].unsqueeze(0)

    # Визуализация внимания с помощью head_view
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    head_view(attention, tokens)

    # Визуализация всего слоя модели
    model_view(attention, tokens)


In [ ]:
import random
import matplotlib.pyplot as plt

def plot_attention_for_random_samples(val_df, val_df_eng, model_ru, model_math, tokenizer_ru, tokenizer_math, num_samples=3):
    """
    Выбирает случайные примеры из val_df и val_df_eng и визуализирует attention heads

    Args:
        val_df: Russian validation DataFrame
        val_df_eng: English validation DataFrame
        model_ru: Russian model (RuBERT-Tiny2)
        model_math: MathBERT model
        tokenizer_ru: Russian tokenizer
        tokenizer_math: MathBERT tokenizer
        num_samples: Number of samples to analyze
    """

    # Выбираем случайные примеры
    ru_samples = val_df.sample(n=num_samples, random_state=42)
    eng_samples = val_df_eng.sample(n=num_samples, random_state=42)

    print("=== РУССКИЕ ТЕКСТЫ ===")
    for i in range(num_samples):
        bertviz_plot(ru_samples.iloc[i]['Задача'], tokenizer_ru, model_ru)

    print("\n=== АНГЛИЙСКИЕ ТЕКСТЫ ===")
    for i in range(num_samples):
        bertviz_plot(eng_samples.iloc[i]['problem_text'], tokenizer_eng, model_math)


### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
# Запуск анализа
plot_attention_for_random_samples(
    val_df=val_df,
    val_df_eng=val_df_eng,
    model_ru=rubert_tiny_finetuned_with_freezed_backbone,
    model_math=MathBert_finetuned_with_freezed_backbone,
    tokenizer_ru=tokenizer,
    tokenizer_math=tokenizer_eng,
    num_samples=1  # по 1 примеру из каждого датафрейма
)

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE

# Возьму полностью рамороженные дообученные модели
plot_attention_for_random_samples(
    val_df=val_df,
    val_df_eng=val_df_eng,
    model_ru=rubert_tiny_full_finetuned,
    model_math=MathBert_full_finetuned,
    tokenizer_ru=tokenizer,
    tokenizer_math=tokenizer_eng,
    num_samples=1  # по 1 примеру из каждого датафрейма
)